In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import dateparser

random_seed = np.random.seed(1620)

## Data Preparation: Cleaning and Formatting

In [ ]:
def load_data(dataset_path, col_names, null_values):
    return pd.read_csv(dataset_path, sep="\t", names=col_names, 
                     na_values=null_values)

def parse_date(df, col_names, date_parser):
    parsed_dates = df[col_names].dropna().apply(date_parser)
    df[col_names] = pd.to_datetime(parsed_dates, errors='coerce', utc=True)
    return df

def reset_index(df):
    return df.reset_index(drop=True)

def drop_unnecessary_cols(df, col_names):
    return df.drop(columns=col_names)

def clean_null_values(df, col_names):
    return df.dropna(subset=col_names) # clean NAN values

def generate_random_reasons(df, reasons):
    new_reasons=np.random.choice(reasons,size=df['reason'].isna().sum()) # generate random sample of justifications
    df = df.assign(reason=new_reasons)
    
    return df

def generate_random_offset_dates_from(df, from_col, to_col, exp_distribution_param=1.5, time_offset_unit='day'):
    na_claim_date = df[to_col].isna() # check nan claimDate

    random_sample = np.random.exponential(exp_distribution_param,size=na_claim_date.sum()) # generate random sample, with # samples == # nan values
    day_offsets = pd.to_timedelta(random_sample, unit=time_offset_unit) # convert random sample to days offsets

    new_claim_dates = df[from_col][na_claim_date] - day_offsets # generate new claimDate values from publishedDate offsetted with random days

    df[to_col] = df[to_col].fillna(new_claim_dates) # replace only na claimDate values with the new generated dates
    
    return df

def map_labels_to_rating(df, label_col, mapping):
    df[label_col] = df[label_col].map(mapping)
    
    return df

def list_from_string(df, col_name):
    df[col_name] = df[col_name].apply(lambda val: val.strip("[]").replace("'",'').replace('"','').lower().split(", "))
    return df

def map_column_type(df, types_mapping):
    return df.astype(types_mapping)

In [ ]:
dataset_path = Path("train.tsv")
metadata_cols = ["claimID", "claim", "label", "claimURL",
           "reason", "categories", "speaker", "checker", 
           "tags", "articleTitle", "publishDate", "claimDate", "entities"]

mandatory_cols = ["articleTitle","checker","speaker","claim","publishDate","tags","entities"] # minimal columns subset that must not contain any null in any case
justification_taxonomy = ["Missing Context","Deceptive Editing", "Malicious Transformation"]

rating_mapping = { 'false': 'False',
                   'none': 'False',
                   'unsupported': 'False',
                   'no evidence': 'Mostly False',
                   'not the whole story': 'Mostly False',
                   'distorts the facts' : 'Mostly False',
                   'spins the facts' : 'Mostly False',
                   'misleading' : 'Mostly True',
                   'cherry picks' : 'Mostly True',
                   '': 'True'}

column_type_mapping = { "claimID": "string",
                        "claim": "string",
                        "label": "category",
                        "claimURL": "string",
                        "reason": "category",
                        "speaker": "string",
                        "checker": "string",
                        "articleTitle": "string"
}
# PIPELINE 
df = (
    load_data(dataset_path, col_names=metadata_cols, null_values=["None","['None']"])
    .pipe(drop_unnecessary_cols, ['categories'])
    .pipe(clean_null_values, mandatory_cols)
    .pipe(parse_date, "publishDate", dateparser.parse)
    .pipe(parse_date, "claimDate", dateparser.parse)
    .pipe(generate_random_reasons, reasons=justification_taxonomy) # fill reason col with fake data
    .pipe(generate_random_offset_dates_from, from_col='publishDate', to_col='claimDate') # fill claimDate col with fake data
    .pipe(map_labels_to_rating, label_col='label', mapping=rating_mapping) # map label column with our standard 4-values rating systems
    .pipe(list_from_string, col_name='tags') # parse string representation of tags into an actual list of tags
    .pipe(list_from_string, col_name='entities') # parse string representation of entities into an actual list of entities
    .pipe(map_column_type, column_type_mapping)
    .pipe(reset_index)
)

## Pandas to SQL helper functions

In [ ]:
import itertools

def extract_unique_values(column):
    if isinstance(column.dtype, pd.CategoricalDtype):
        values = column.cat.categories.values 
    elif isinstance(column.dtype, pd.StringDtype):
        values = set(column.values)
    else: # for object dtype
        values = itertools.chain.from_iterable(column.values) # extract column values from df and flatten it out
        values = set(values) # make unique
    return list(values)
        
def swap_index_with_col(df, col_name, id_col_name):
    """
    swap the index of the input dataframe (df) with the col_name column.
    This is an helper function for creating a lookup table.
     
    """
    new_df = df.copy()
    new_df[id_col_name] = new_df.index
    
    return new_df.set_index(col_name)

def generate_fk(fk_col, parent_table, parent_col_name):
    """
    Generate a fk column looking into a lookup table.
    The values inside fk_col are replaced with the corrisponding value index in the parent_table.
    parent_col_name is the column in the parent table where lookup takes place.
    
    eg. fk_col = ['b','c','a']
        parent_table = [[1,'a'],['2','b'],[3,'c']
        function output is a  new fk_col = [2,3,1]
    """
    id_col = 'id'
    lookup = swap_index_with_col(parent_table, 
                                 parent_col_name,
                                 id_col_name=id_col) # make a lookup table with parent_col_name as index and id_col as value
    if isinstance(fk_col.dtype, pd.StringDtype):
        new_fk_col = fk_col.apply(lambda value: lookup.loc[value,id_col]) # replace names with integer id
    else: # for column with list-like elements
        new_fk_col = fk_col.apply(lambda values: [lookup.loc[val,id_col] for val in values]) # replace names with integer id
    
    return new_fk_col

def generate_joint_table(df_master, join_col, left_table, right_table, lookup_col, left_pk, right_pk):
    
    joint = pd.DataFrame({left_pk: left_table.index,
                          right_pk: generate_fk(df_master[join_col],right_table, lookup_col)})
    
    return joint.explode(right_pk, ignore_index=True)

## Tags Table

In [ ]:
tag_table = pd.DataFrame({'label': extract_unique_values(df['tags'])})
topic_joint_table = generate_joint_table(df, 'tags', df['claimURL'], tag_table, 'label', 'review_id', 'tag_id')

## External Entity Table

In [ ]:
external_entity_table = pd.DataFrame({'label': extract_unique_values(df['entities'])})
mention_joint_table = generate_joint_table(df, 'entities', df['claimURL'], external_entity_table, 'label', 'review_id', 'entity_id')

## Agent Table

In [ ]:
## Claimants agents
claimants = pd.DataFrame({'name': extract_unique_values(df['speaker']),
                          'type': 'person',
                          'role': 'claimant'
                         })
# some claimants are organizations
organization_filter = claimants['name'].isin(['NRSC','Election TV Ads',
                        'Various websites','FactCheck.org',
                        'Viral Claim', 'National Republican Senatorial Committee',
                        'Senate Majority PAC','Senate Leadership Fund'])

claimants.loc[organization_filter ,'type'] = 'organization' # manual adjustment


## Fact checkers agents
factcheckers = pd.DataFrame({'name': extract_unique_values(df['checker']),
                             'type': 'person',
                             'role': 'factchecker'
                         })

# remove FactCheck.org row because is inserted after with organization group
mask = factcheckers.loc[factcheckers['name'] == 'FactCheck.org'].index
factcheckers.drop(mask)


## Organizations agents
organizations = pd.DataFrame({'name': ['FactCheck.org','Pagella Politica', 'LaVoce.info','PolitiFact', 
                                       'WashingtonPost','International Fact Checking Network',
                                       'Duke Reporter lab'],
                              'type': 'organization',
                              'role': 'factchecker'
                              })
## Software agents
software_bots = pd.DataFrame({'name': ['DeepFakeTwitter','DistoClaim','HAL9000'],
                              'type': 'software',
                              'role': 'claimant'
                              })
# organization MUST be inserted before factchecker due to self referential foreign key
agents = pd.concat([organizations, claimants, factcheckers, software_bots], ignore_index=True)

## Generate random affiliations 
fc_organizations = agents[(agents['type'] == 'organization') & (agents['role'] == 'factchecker')]
fc_person = agents[(agents['type'] == 'person') & (agents['role'] == 'factchecker')]
random_affiliations_indeces = np.random.choice(fc_organizations.index, size=len(fc_person))

agents.loc[fc_person.index,'affiliation_id'] = random_affiliations_indeces

agent_table = agents

## Claim Table

In [ ]:
claims = df[['claim','speaker','claimDate','claimURL']].copy()


claimants = agents[agents['role']=='claimant'].astype({'name':'string'}) # filter only claimant agents


claims.loc[:,'speaker'] = generate_fk(df['speaker'], claimants, 'name') #replace names with id
claims['language'] = 'en'
claims = claims.rename({'claim': 'content', #align to SQL schema
                        'speaker': 'claimant_id',
                        'claimDate': 'publication_date',
                        'claimURL': 'url'}, axis='columns') # WARNING! claimURL is the review article URL, since no data is provided on the claim source url, we used this one

claim_table = claims

## Rating

In [ ]:
import lorem

In [ ]:
rating_systems = ['https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/#Truth-O-Meter%20ratings',
                  'https://pagellapolitica.it/static/metodologia',
                  'https://www.lavoce.info/come-facciamo-il-fact-checking/']


ratings = pd.DataFrame({'value': df['label'],
                        'comment': pd.Series(dtype='string'),
                        'media_url':'',
                        'system_url':np.random.choice(rating_systems,size=len(df['label']))})

# generate fake rating media association (only for Politifact)
associated_media = {'False': 'https://static.politifact.com/img/meter-false.jpg',
                    'Mostly False': 'https://static.politifact.com/img/meter-mostly-false.jpg',
                    'Mostly True': 'https://static.politifact.com/img/meter-mostly-true.jpg',
                    'True': 'https://static.politifact.com/img/meter-mostly-true.jpg'}

politifact_mask = ratings[ratings['system_url']=='https://www.politifact.com/article/2018/feb/12/principles-truth-o-meter-politifacts-methodology-i/#Truth-O-Meter%20ratings'].index
ratings['media_url'] = ratings.iloc[politifact_mask].apply(lambda row: associated_media[row['value']], axis='columns')

# generate fake random comment 
ratings['comment'] = ratings['comment'].apply(lambda x: lorem.sentence())

rating_table = ratings # just for naming consistence

## Justifications

In [ ]:
liarplus_path = Path("LIARPlus_validation.jsonl")
liar_df = pd.read_json(liarplus_path, lines=True)
# random sample justification from liar plus dataset
# WARNING! The sampling procedure is naive, this means that no sense records could be produced
# i.e [Missing Context, "Donald Trump this time is completely right!"]

liarplus_justifications = (liar_df['justification'].sample(n=len(df),random_state=random_seed)
                                                   .reset_index(drop=True))

judgment_table = pd.DataFrame({'label': df['reason'],
                               'justification': liarplus_justifications})


## Review

In [ ]:
reviews = df[['publishDate','claimURL']].copy()

reviews = reviews.rename({'publishDate': 'publication_date',
                          'claimURL': 'url'}, axis='columns')




reviews['title'] = 'XXXX'
reviews['content'] = 'YYYY'
reviews['language'] = 'en'
reviews['rating_id'] = rating_table.index
reviews['judgment_id'] = judgment_table.index




factcheckers = agents[agents['role']=='factchecker'].astype({'name':'string'}) # filter only factcheckers agents

review_author_joint_table = generate_joint_table(df, 'checker', df['claimURL'], factcheckers, 'name', 'review_id', 'agent_id')
about_joint_table = pd.DataFrame({'review_id': reviews.index,
                                  'claim_id': claims.index})



review_table = reviews


In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
#profile = ProfileReport(df, title='Pandas Profiling Report', explorative=True)
#profile.to_file("data_cleaning_report.html")

In [ ]:
#df_orig =  load_data(dataset_path, col_names=metadata_cols, null_values=["None","['None']"])
#profile = ProfileReport(df_orig, title='Pandas Profiling Report', explorative=True)
#profile.to_file("original_data.html")

# Insert Into RDB

In [ ]:
from sqlalchemy import create_engine
from psycopg2.extensions import register_adapter, AsIs

register_adapter(np.int64, AsIs)

engine = create_engine("postgresql://{username}:{password}@{host}:{port}/{database}".format(username='modsem',
                                                                                            password='modsem',
                                                                                            host='localhost',
                                                                                            port=5432,
                                                                                            database='faktnews'))

In [ ]:
tag_table.to_sql('tag',con=engine, if_exists='append', index_label='id')

In [ ]:
rating_table.to_sql('rating',con=engine, if_exists='append', index_label='id')

In [ ]:
judgment_table.to_sql('judgment',con=engine, if_exists='append', index_label='id')

In [ ]:
review_table.to_sql('review',con=engine, if_exists='append', index_label='id')

In [ ]:
agent_table.to_sql('agent',con=engine, if_exists='append', index_label='id',method=None)

In [ ]:
claim_table.to_sql('claim',con=engine, if_exists='append', index_label='id')

In [ ]:
external_entity_table.to_sql('external_entity',con=engine, if_exists='append', index_label='id')

In [ ]:
topic_joint_table.to_sql('topic',con=engine, if_exists='append', index_label='id')

In [ ]:
mention_joint_table.to_sql('mention',con=engine, if_exists='append', index_label='id')
review_author_joint_table.to_sql('review_author',con=engine, if_exists='append', index_label='id')
about_joint_table.to_sql('about',con=engine, if_exists='append', index_label='id')